In [1]:
import yaml

#讀取yaml檔
with open(r'project_yml/sql_columns_0525.yml',encoding="utf-8") as file:
    list_full = yaml.load(file, Loader=yaml.FullLoader)

with open(r'project_yml/HDFS_columns_variables_52.yml',encoding="utf-8") as file:
    list_52 = yaml.load(file, Loader=yaml.FullLoader)
    print(list_52)


{'selected_columns': ['Current_Ratio', 'oper_cashflow_liab', 'current_liab_current_assets', 'working_capital_assets', 'Net_Sales_Assets', 'Receivables_Turnover_Ratio', 'Inventory_Turnover', 'Fixed_Assets_Turnover', 'net_worth_turnover_ratio', 'INV_AR_Equity', 'l16', 'l17', 'l18', 'Rate_of_RD_expenses', 'Revenue_Growth_Rate', 'Operating_gross_profit_growth_rate', 'Net_profit_growth_rate_before_tax', 'l10', 'l11', 'Gross_margin', 'realized_gross_profit_on_sales', 'Operating_margin', 'earning_before_tax_margin', 'Operating_Expense_Ratio', 'Total_Non_Op_Income_Exp', 'Net_Asset_Value_per_Share_F', 'l2', 'l3', 'l4', 'Return_on_Operating_Assets', 'net_income_change_rate', 'Retained_earnings_assets', 'Income_Before_Tax_and_Interests_assets', 'debt_ratio', 'Interest_Coverage_Ratio', 'Cash_flow_ratio', 'ration_of_interest_expense', 'l1', 'Debt_Equity_Ratio', 'Equity_Asset_Ratio', 'Ratio_of_liab_to_assets', 'Debt_Eqity_rate', 'i69', 'i71', 'fin_depen_related_parties', 'ratio_invest_equity', 'rati

In [2]:
import pandas as pd
import numpy as np
import re

perfdf = pd.read_csv("data\\merge_data.csv",encoding="UTF-8" ,low_memory=False)



In [3]:
#將 "-" ,"" 轉成 nan

perfdf = perfdf.applymap(lambda x : x.replace(" ","") if type(x) == str else x )
perfdf.replace({'-':np.nan,"":np.nan,True:1,False:0,"True":1,"False":0,},inplace=True)


In [4]:
#將欄位轉為英文
columns = []
for i in perfdf.columns:
    try:
        data = list_full['rename_columns'][i.strip()]
        columns.append(data)
    except:
        if i.strip() == "庫藏股數（母持及子持母）":
            columns.append('Total_Treasury_Stock')
        else:
            columns.append(i.strip())

perfdf.columns = columns 

In [5]:
#計算比率
perfdf["i83"] = perfdf["i83"].fillna("0")

perfdf["oper_cashflow_liab"] = perfdf["Cash_Flow_Oper"].astype('float')/ perfdf["Total_Liab"].astype('float')

perfdf["current_liab_current_assets"] = perfdf["Total_Current_Liab"].astype('float')/ perfdf["Total_Current_Assets"].astype('float')

perfdf["working_capital_assets"] = perfdf["working_capital"].astype('float')/ perfdf["Total_Assets"].astype('float')

perfdf["Net_Sales_Assets"] = perfdf["Net_Sales"].astype('float')/ perfdf["Total_Assets"].astype('float')

perfdf["Retained_earnings_assets"] = perfdf["Retained_earnings"].astype('float')/ perfdf["Total_Assets"].astype('float')

perfdf["Income_Before_Tax_and_Interests_assets"] = perfdf["Income_Before_Tax_and_Interests"].astype('float')/ perfdf["Total_Assets"].astype('float')

perfdf["fin_depen_related_parties"] = perfdf["i83"].astype('float')/ perfdf["Long_short_term_loans"].astype('float')

perfdf["ratio_invest_equity"] = perfdf["Investment_Equity_Method"].astype('float')/ perfdf["Total_Equity"].astype('float')

perfdf["ratio_invest_assets"] = perfdf["Investment_Equity_Method"].astype('float')/ perfdf["Total_Assets"].astype('float')


perfdf["Net_Asset_Value_per_Share_F"]   = perfdf["Total_Equity"].astype('float') / perfdf["weighted_average_shares"].astype('float')

In [6]:
perfdf["ratio_director_shareholder_holding"] = perfdf["w9"].str.replace(",","").astype(float)/perfdf["weighted_average_shares"].astype(float)*1000

perfdf["ratio_director_shareholder_Pledge"] = perfdf["w8"].str.replace("%","").astype(float)/100

perfdf["ratio_majority shareholder"] = perfdf["w10"].astype(float)/perfdf["weighted_average_shares"].astype(float)*1000

perfdf["ratio_endorsement_equity"] = perfdf["w11"].str.replace(",","").astype(float)/perfdf["Total_Equity"].astype(float)

perfdf["adj_laib_equity"] = (perfdf["Total_Liab"].astype('float')/perfdf["Total_Equity"].astype('float'))*(1+perfdf["w8"].str.replace("%","").astype(float))


In [7]:
perfdf.head()

,company,abbr,listing_type,TSE_industry,IPO_date,code,calendar_month,tax_id_number,securities_number,phone_number,...,Retained_earnings_assets,Income_Before_Tax_and_Interests_assets,fin_depen_related_parties,ratio_invest_equity,ratio_invest_assets,ratio_director_shareholder_holding,ratio_director_shareholder_Pledge,ratio_majority shareholder,ratio_endorsement_equity,adj_laib_equity
0,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.034284,0.019475,0.035276,0.579136,0.197084,58387.011145,0.5204,0.0,0.315321,102.819650
1,1102,亞泥,TSE,01,1962/06/18,1102,12,ID03244509,TW0001102002,02-27338000,...,0.113096,0.020028,0.000000,0.843143,0.422366,320932.582757,0.3009,0.0,0.208490,30.973089
2,1103,嘉泥,TSE,01,1969/11/27,1103,12,ID11892801,TW0001103000,02-25515211,...,0.035452,0.038463,0.012615,0.167757,0.054445,123557.696762,0.0193,0.0,0.025863,6.098004
3,1104,環泥,TSE,01,1971/02/02,1104,12,ID07568009,TW0001104008,02-25077801,...,0.099562,-0.002999,0.000000,0.515537,0.274526,119594.990821,0.0737,0.0,0.071384,7.348158
4,1107,建台,NaN,01,2007/10/22,1107,12,ID03322504,TW0001107001,07-5666299,...,-0.155564,-0.008556,0.000000,0.250359,0.111275,NaN,NaN,NaN,NaN,NaN


In [8]:
#處理net income ifrs 改制合併問題

perfdf[["Total_compre_income","Net_Income"]] = perfdf[["Total_compre_income","Net_Income"]].applymap(lambda x : np.nan if x == "nan" else x) 

# 主要邏輯 如果兩個欄位只有一個有值，就用有值的補
# 如果兩個都有值，就用新制 Total_compre_income補
# 都沒值就 nan

# 創一個欄位叫 net_income_combine 將兩欄 isnull()相加，如果為0代表兩欄都有值 如果為1代表一欄有值 為2代表兩欄都沒值 
perfdf["net_income_combine"] = perfdf[["Total_compre_income","Net_Income"]].isnull().sum(axis =1)

#net_income_combine為0，用Total_compre_income補
perfdf.loc[ perfdf["net_income_combine"] == 0 ,"new_net_income"] = perfdf.loc[ perfdf["net_income_combine"] == 0 ,"Total_compre_income"]

#net_income_combine為1 Total_compre_income為nan的用Net_Income補
perfdf.loc[ (perfdf["net_income_combine"] == 1) & (perfdf["Total_compre_income"].isnull()) ,"new_net_income"]  = perfdf.loc[ (perfdf["net_income_combine"] == 1) & (perfdf["Total_compre_income"].isnull()) ,"Net_Income"]  

#net_income_combine為1 Net_Income為nan的用Total_compre_income補
perfdf.loc[ (perfdf["net_income_combine"] == 1) & (perfdf["Net_Income"].isnull()) ,"new_net_income"]  = perfdf.loc[ (perfdf["net_income_combine"] == 1) & (perfdf["Net_Income"].isnull()) ,"Total_compre_income"]  

#net_income_combine為1 補nan
perfdf.loc[ perfdf["net_income_combine"] == 2 ,"new_net_income"] = np.nan


In [9]:
perfdf["time_q"] = perfdf[["company","year","mounth","new_net_income"]].apply(lambda x : pd.to_datetime(str(x["year"])+str(x["mounth"]) , format='%Y%m',errors='ignore').to_period('Q')  ,axis =1)


In [10]:

def net_income(x):
    x.index = x["time_q"]
    t = x["new_net_income"].shift(1,freq="q")  
    data = pd.merge(x ,t , left_index = True ,right_index = True ,how ="left")
    return data.reset_index(drop = True)
    
perfdf = perfdf.groupby("company").apply(net_income)

In [11]:
perfdf["net_income_change_rate"] = (perfdf["new_net_income_x"].astype(float) - perfdf["new_net_income_y"].astype(float)) /  perfdf["new_net_income_y"].astype(float)*100

In [12]:
perfdf = perfdf.reset_index(drop = True)


In [14]:
# 尋找近兩年淨利為負
def search(x):
    x.index = x["time_q"]
    col =[4,8]
    for i ,q in enumerate(col):
        df = pd.DataFrame()
        df["w1_past_"+str(i)] = x["w1"].shift(q,freq="q")
        x = pd.merge(x ,df["w1_past_"+str(i)] , left_index = True ,right_index = True ,how ="left")
    x["negative_two_years"] = (x["w1_past_0"].str.replace(",","").astype(float)<0)&(x["w1_past_1"].str.replace(",","").astype(float)<0)
    x["negative_two_years"] = x["negative_two_years"].astype(int)
#     x.loc[(x["本期淨利_past_0"].str.replace(",","").astype(float)>0)&(x["本期淨利_past_1"].str.replace(",","").astype(float)>0),"negative_two_years"] = 0
    return x
perfdf = perfdf.groupby("company").apply(search)

In [19]:
perfdf = perfdf.reset_index(drop = True)


0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
124577    NaN
124578    NaN
124579    NaN
124580    NaN
124581    NaN
Name: crisis_occurrence_date, Length: 124582, dtype: object

In [21]:
#危機事件標記

#定義季別滑動 丟進危機年月 回傳過去四季季別(包括當季)
def crisis_move(year, mounth):
    if int(mounth) <= 3:
        return [[int(year) , 3],[int(year)-1 , 12],[int(year)-1 , 9],[int(year) - 1, 6]]
    if int(mounth) > 3 and int(mounth) <= 6:
        return [[int(year) , 6],[int(year) , 3],[int(year) - 1, 12],[int(year) - 1, 9]]
    if int(mounth) > 6 and int(mounth) <= 9:
        return [[int(year) , 9],[int(year) , 6],[int(year) , 3 ],[int(year) - 1, 12]]
    if int(mounth) >= 10:
        return [[int(year), 12],[int(year) , 9],[int(year) - 1, 6],[int(year) - 1, 3]]

# 標記危機發生日的前四季為1
def find_crisis(x):
    #將危機發生日找出 並去重成list
    x["crisis_occurrence_date"] = x.apply(lambda x : "nan" if re.match("..../../..",str(x["crisis_occurrence_date"])) == None else x["crisis_occurrence_date"] ,axis = 1)
    d = x["crisis_occurrence_date"].drop_duplicates().values.tolist()
    #如果沒有nan 代表有危機發生
    if "nan" not in d :

        # 遍歷所有危機發生日
        for i in d :
            date = re.match("(....)/(..)/..", i)
            #將危機發生日帶入 產出過去四季
            year_mounth = crisis_move(date.group(1), date.group(2))
            x["crisis"] = 0

            #找巡那四季的 rows 位置 並標記為1
            for i in year_mounth:
                try:
                    location = x.index[(x["year"] == i[0]) & (x["mounth"] == i[1])].tolist()
                    x.loc[location[0], "crisis"] = 1
                except:
                    pass
            return x
    else:
        x["crisis"] = 0
        return x
# 以公司別分類 套用以上方法
data = perfdf.groupby("company").apply(find_crisis)


In [24]:
perfdf["crisis_occurrence_date"]

0         nan
1         nan
2         nan
3         nan
4         nan
         ... 
124577    nan
124578    nan
124579    nan
124580    nan
124581    nan
Name: crisis_occurrence_date, Length: 124582, dtype: object

In [27]:
#標記危機發生類別
def extract(x):
    data = re.match("([\u4e00-\u9fa5]*)\((....)/(..)/..\)", str(x["explanation_of_crisis_2"]))
    if data != None:
        x["crsis_type"] = data.group(1)
    else:
        x["crsis_type"] = np.nan
    return x
data2 =  data.apply(extract,axis=1)

#將景氣不加停工移除
raw_data = data2.drop(data2.loc[data2["crsis_type"]== "景氣不佳停工",:].index)


In [28]:
raw_data.replace("nan",np.nan,inplace = True)

In [29]:

#選取出要丟入模型的變數
train_data = raw_data[list_52["selected_columns"]+["negative_two_years","crisis"]]
train_data.head()


,Current_Ratio,oper_cashflow_liab,current_liab_current_assets,working_capital_assets,Net_Sales_Assets,Receivables_Turnover_Ratio,Inventory_Turnover,Fixed_Assets_Turnover,net_worth_turnover_ratio,INV_AR_Equity,...,fin_depen_related_parties,ratio_invest_equity,ratio_invest_assets,ratio_endorsement_equity,adj_laib_equity,ratio_director_shareholder_holding,ratio_majority shareholder,ratio_director_shareholder_Pledge,negative_two_years,crisis
0,82.01,0.049489,1.219379,-0.019934,0.199249,5.87,11.06,0.32,0.61,16.60,...,0.035276,0.579136,0.197084,0.315321,102.819650,58387.011145,0.0,0.5204,0,0
1,69.77,0.009297,1.433267,-0.055556,0.245169,6.34,11.76,0.39,0.71,16.44,...,0.000000,0.516496,0.186435,0.274478,103.407894,70659.257213,0.0,0.5741,0,0
2,95.36,0.126727,1.048635,-0.006211,0.314044,7.04,16.28,0.48,0.82,14.86,...,0.000000,0.440749,0.169755,0.298985,74.311318,79575.538078,0.0,0.4555,0,0
3,139.54,0.114485,0.716633,0.045026,0.345147,7.43,16.37,0.54,0.86,16.57,...,0.000000,0.374848,0.154459,0.294469,67.760844,77062.331098,0.0,0.4649,0,0
4,143.47,0.097925,0.696989,0.062481,0.194898,8.13,12.18,0.61,0.97,19.64,...,0.000000,0.049468,0.020088,0.281849,60.068678,78608.659872,0.0,0.4007,0,0


In [30]:
train_data_ = train_data.replace([np.inf, -np.inf], np.nan)
train_data_  = train_data_.round(2)


In [31]:
train_data_["company"] = raw_data["company"]
train_data_.head()

,Current_Ratio,oper_cashflow_liab,current_liab_current_assets,working_capital_assets,Net_Sales_Assets,Receivables_Turnover_Ratio,Inventory_Turnover,Fixed_Assets_Turnover,net_worth_turnover_ratio,INV_AR_Equity,...,ratio_invest_equity,ratio_invest_assets,ratio_endorsement_equity,adj_laib_equity,ratio_director_shareholder_holding,ratio_majority shareholder,ratio_director_shareholder_Pledge,negative_two_years,crisis,company
0,82.01,0.05,1.22,-0.02,0.20,5.87,11.06,0.32,0.61,16.60,...,0.58,0.20,0.32,102.82,58387.01,0.0,0.52,0,0,1101
1,69.77,0.01,1.43,-0.06,0.25,6.34,11.76,0.39,0.71,16.44,...,0.52,0.19,0.27,103.41,70659.26,0.0,0.57,0,0,1101
2,95.36,0.13,1.05,-0.01,0.31,7.04,16.28,0.48,0.82,14.86,...,0.44,0.17,0.30,74.31,79575.54,0.0,0.46,0,0,1101
3,139.54,0.11,0.72,0.05,0.35,7.43,16.37,0.54,0.86,16.57,...,0.37,0.15,0.29,67.76,77062.33,0.0,0.46,0,0,1101
4,143.47,0.10,0.70,0.06,0.19,8.13,12.18,0.61,0.97,19.64,...,0.05,0.02,0.28,60.07,78608.66,0.0,0.40,0,0,1101


In [33]:

raw_data.head()


,company,abbr,listing_type,TSE_industry,IPO_date,code,calendar_month,tax_id_number,securities_number,phone_number,...,ratio_endorsement_equity,adj_laib_equity,net_income_combine,new_net_income_x,time_q,new_net_income_y,w1_past_0,w1_past_1,negative_two_years,crisis
0,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.315321,102.819650,1,190575,2001Q4,NaN,NaN,NaN,0,0
1,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.274478,103.407894,1,286737,2002Q4,NaN,"190,575.00",NaN,0,0
2,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.298985,74.311318,1,1762182,2003Q4,NaN,"286,737.00","190,575.00",0,0
3,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.294469,67.760844,1,4421526,2004Q4,NaN,"1,762,182.00","286,737.00",0,0
4,1101,台泥,TSE,01,1962/02/09,1101,12,ID11913502,TW0001101004,02-25317099,...,0.281849,60.068678,2,NaN,2005Q2,NaN,NaN,NaN,0,0


In [34]:
raw_data.to_csv("data/raw_data.csv",index= False)

In [ ]:
train_data_.head()

In [ ]:
train_data_.agg(["max","min"])

In [ ]:
#將 nan 補為-9999
r_forest_data = train_data_.fillna(-9999)
xgboost_data = train_data_

In [ ]:
r_forest_data.isnull().sum()


In [ ]:
xgboost_data.isnull().sum()

In [ ]:
#危機

r_forest_data.crisis.sum() , len(r_forest_data.crisis)-r_forest_data.crisis.sum()

In [ ]:
#危機公司空值數
train_data_.loc[train_data["crisis"] == 1 ,:].isnull().sum()

In [ ]:
#非危機公司空值數
train_data_.loc[train_data["crisis"] == 0 ,:].isnull().sum()

In [ ]:
r_forest_data.to_csv("data/random_tarin.csv")

In [ ]:
xgboost_data.to_csv("data/xgboost_train.csv")